In [25]:
import gspread
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import os

def auth_gspread():
    try:
        gc = gspread.service_account(filename='gfile.json')
        sheet = gc.open("cmacc_test").sheet1
        print("Successfully authenticated and accessed the sheet.")
        return sheet
    except Exception as e:
        print(f"Error authenticating Google Sheets: {e}")
        raise

def prepare_for_google_sheets(df):
    # Fill NaN values in numeric columns with a default value or None
    df = df.copy()  # Create a copy to avoid setting on a slice warning
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].fillna(0)  # Fill numeric NaNs with 0
    non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns
    df[non_numeric_cols] = df[non_numeric_cols].apply(lambda x: x.fillna(''))  # Fill non-numeric NaNs with empty string
    return df

def upload_to_google_sheets(df):
    sheet = auth_gspread()
    sheet.clear()  # Clears the existing content in the sheet
    # Convert only non-numeric columns to string
    df = df.apply(lambda col: col.astype(str) if col.name not in ['age_numeric', 'Weight_num', 'Days_at_shelter'] else col)
    # Upload the data
    try:
        # Ensure that you send data to Google Sheets in batches if too large
        values = [df.columns.values.tolist()] + df.values.tolist()
        sheet.update(values)
        print("Data saved to Google Sheet.")
    except Exception as e:
        print(f"Error updating Google Sheet: {e}")

def assign_result(row, all_days_df):
    current_date = row["Scrape Date"]
    dog_id = row["ID"]
    most_recent_date = all_days_df["Scrape Date"].max()
    is_in_next = all_days_df[(all_days_df["Scrape Date"] > current_date) & (all_days_df["ID"] == dog_id)].shape[0] > 0
    return "Adopted" if not is_in_next and current_date != most_recent_date else "Looking for Furever Home"

def convert_age_to_years(age):
    if isinstance(age, str):
        parts = age.split(',')
        total_years = 0
        for part in parts:
            number, unit = part.strip().split()[:2]  # Split each part and get the first two elements (number and unit)
            number = int(number)  # Convert the number to integer
            if 'year' in unit:
                total_years += number  # Add the number directly to years
            elif 'month' in unit:
                total_years += number / 12  # Convert months to fraction of year and add
            elif 'week' in unit:
                total_years += number / 52  # Convert weeks to fraction of year and add
        return round(total_years, 2)
    return None


def wrangle(dogs_scrape):
    # Convert date columns to datetime only once
    for col in ['Brought to Shelter', 'Scrape Date']:
        dogs_scrape[col] = pd.to_datetime(dogs_scrape[col], errors='coerce').dt.date
    dogs_scrape['age_numeric'] = dogs_scrape['Age'].apply(convert_age_to_years)
    dogs_scrape['Weight_num'] = dogs_scrape['Weight'].str.extract(r'(\d+\.?\d*)').astype(float)
    dogs_scrape.rename(columns={'Name': 'Name_ID'}, inplace=True)
    dogs_scrape['Name'] = dogs_scrape['Name_ID'].str.extract(r'([A-Za-z\s]+) \(')
    dogs_scrape['ID'] = dogs_scrape['Name_ID'].str.extract(r'(\d+)').astype(float)
    dogs_scrape["result"] = dogs_scrape.apply(assign_result, axis=1, all_days_df=dogs_scrape)
    dogs_scrape['Days_at_shelter'] = (dogs_scrape['Scrape Date'] - dogs_scrape['Brought to Shelter']).apply(lambda x: x.days if pd.notnull(x) else None).astype(float)
    dogs_scrape['location_simple'] = np.where(
    dogs_scrape['Kennel Location'].str.contains('toom', case=False, na=False), 'Toomey',
    np.where(
        dogs_scrape['Kennel Location'].str.contains('CARE|LST|ADOPT|PUPPY|INTAKE|CLINIC|FOUND|FERRET|ADPT', case=False), 'Byrum',
    np.where(
        dogs_scrape['Kennel Location'].str.contains('Foster', case=False), 'Foster',
         dogs_scrape['Kennel Location'])))
    dogs_scrape['kennel_name'] = dogs_scrape['Kennel Location'].str.extract(r'([A-Za-z\s]+)')  # Extracts name/hallyway location
    dogs_scrape['kennel_num'] = dogs_scrape['Kennel Location'].str.extract(r'(\d+)')  # Extracts number
    dogs_scrape['side'] = dogs_scrape['Kennel Location'].apply(
    lambda x: '' if 'FOSTER' in x or x[-1] not in ['R', 'L'] else x[-1])
    return dogs_scrape

def fetch_and_parse(url):
    try:
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        return BeautifulSoup(response.text, 'html.parser')
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"Request error occurred: {req_err}")
    return None

def extract_data(soup):
    if soup is None:
        return []
    dogs = soup.find_all('div', class_='gridResult')
    dog_list = []
    for dog in dogs:
        try:
            dog_list.append({
                'Name_ID': dog.find('span', class_='text_Name results').text.strip() if dog.find('span', class_='text_Name results') else '',
                'Gender': dog.find('span', class_='text_Gender results').text.strip() if dog.find('span', class_='text_Gender results') else '',
                'Breed': dog.find('span', class_='text_Breed results').text.strip() if dog.find('span', class_='text_Breed results') else '',
                'Age': dog.find('span', class_='text_Age results').text.strip() if dog.find('span', class_='text_Age results') else '',
                'Animal Type': dog.find('span', class_='text_Animaltype results').text.strip() if dog.find('span', class_='text_Animaltype results') else '',
                'Weight': dog.find('span', class_='text_Weight results').text.strip() if dog.find('span', class_='text_Weight results') else '',
                'Brought to Shelter': dog.find('span', class_='text_Broughttotheshelter results').text.strip() if dog.find('span', class_='text_Broughttotheshelter results') else '',
                'Kennel Location': dog.find('span', class_='text_KennelLocation results').text.strip() if dog.find('span', class_='text_KennelLocation results') else ''
            })
        except AttributeError as e:
            print(f"Error extracting data for one dog: {e}")
    return dog_list

def scrape_dog_data():
    base_url = 'https://24petconnect.com/CLTAdopt'
    index = 0
    all_dogs = []
    while True:
        current_url = f"{base_url}?index={index}&at=DOG"
        print(f"Fetching data from index: {index}")
        soup = fetch_and_parse(current_url)
        if soup is None:
            print("Error fetching page. Exiting loop.")
            break
        new_dogs = extract_data(soup)
        if not new_dogs:
            print("No more dogs found. Exiting loop.")
            break
        all_dogs.extend(new_dogs)
        index += 30
        time.sleep(1)
    if all_dogs:
        dogs_at_shelter = pd.DataFrame(all_dogs)
        dogs_at_shelter['Scrape Date'] = pd.to_datetime('today').strftime('%Y-%m-%d')
        dogs_at_shelter = wrangle(dogs_at_shelter)
        csv_file = 'shelterdata.csv'
        csv_file_new = 'dogs_at_shelter_test.csv'
        if os.path.exists(csv_file):
            existing_data = pd.read_csv(csv_file)
            filtered_data = existing_data[existing_data['Scrape Date'] != pd.to_datetime('today').strftime('%Y-%m-%d')]
            updated_data = pd.concat([filtered_data, dogs_at_shelter], ignore_index=True)
            updated_data = prepare_for_google_sheets(updated_data)
            updated_data['Days_at_shelter'] = pd.to_numeric(updated_data['Days_at_shelter'], errors='coerce')
            updated_data['Days_at_shelter'].fillna('', inplace=True)
            updated_data['ID'] = pd.to_numeric(updated_data['ID'], errors='coerce')
            updated_data['ID'].fillna('', inplace=True)
            updated_data.to_csv(csv_file, index=False)
            try:
                sheet = auth_gspread()
                sheet.clear()

                # Apply conversion: only non-numeric columns to string
                for col in updated_data.columns:
                    if updated_data[col].dtype == np.number:
                        continue  # Skip the conversion to string for numeric columns
                    else:
                        updated_data[col] = updated_data[col].astype(str)  # Convert non-numeric columns to strings

                # Prepare the data as list of lists, each sublist corresponding to a row
                data_to_upload = [updated_data.columns.values.tolist()] + updated_data.values.tolist()
                
                # Update Google Sheet
                sheet.update(data_to_upload)
                print("Data saved to Google Sheet.")
            except Exception as e:
                print(f"Error updating Google Sheet: {e}")
        else:
            dogs_at_shelter.to_csv(csv_file_new, index=False)
            print("Data saved as new file.")




scrape_dog_data()



Fetching data from index: 0
Fetching data from index: 30
Fetching data from index: 60
Fetching data from index: 90
Fetching data from index: 120
Fetching data from index: 150
Fetching data from index: 180
Fetching data from index: 210
Fetching data from index: 240
Fetching data from index: 270
Fetching data from index: 300
Fetching data from index: 330
HTTP error occurred: 500 Server Error: Internal Server Error for url: https://24petconnect.com/CLTAdopt?index=330&at=DOG
Error fetching page. Exiting loop.


/var/folders/5l/7r0lrfyj4dn5flz3ntwmqs380000r6/T/ipykernel_6173/1841312336.py:152: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  updated_data['Days_at_shelter'].fillna('', inplace=True)
/var/folders/5l/7r0lrfyj4dn5flz3ntwmqs380000r6/T/ipykernel_6173/1841312336.py:154: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting v

Successfully authenticated and accessed the sheet.


/var/folders/5l/7r0lrfyj4dn5flz3ntwmqs380000r6/T/ipykernel_6173/1841312336.py:162: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if updated_data[col].dtype == np.number:
/var/folders/5l/7r0lrfyj4dn5flz3ntwmqs380000r6/T/ipykernel_6173/1841312336.py:162: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if updated_data[col].dtype == np.number:
/var/folders/5l/7r0lrfyj4dn5flz3ntwmqs380000r6/T/ipykernel_6173/1841312336.py:162: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. The current result is `float64` which is not strictly correct.
  if updated_data[col].dtype == np.number:
/var/folders/5l/7r0lrfyj4dn5flz3ntwmqs380000r6/T/ipykernel_6173/1841312336.py:162: DeprecationWarning: Converting `np.inexact` or `np.floating` to a dtype is deprecated. Th

Data saved to Google Sheet.


In [55]:
import schedule

# Scheduling
def schedule_job():
    schedule.every(30).minutes.do(scrape_dog_data)
    print("Scheduled job to run every 30 mins...")
    try:
        while True:
            schedule.run_pending()
            time.sleep(1)
    except KeyboardInterrupt:
        print("Script interrupted by user, exiting.")


if __name__ == '__main__':
    scrape_dog_data()
    schedule_job()  # Then start the scheduled job



Fetching data from index: 0
Fetching data from index: 30
Fetching data from index: 60
Fetching data from index: 90
Fetching data from index: 120
Fetching data from index: 150
Fetching data from index: 180
Fetching data from index: 210
Fetching data from index: 240
Fetching data from index: 270
Fetching data from index: 300
Fetching data from index: 330
HTTP error occurred: 500 Server Error: Internal Server Error for url: https://24petconnect.com/CLTAdopt?index=330&at=DOG
Error fetching page. Exiting loop.
Total dogs fetched: 309
Existing data before filtering: (10815, 21)
Data after filtering out today's date: (10506, 21)
Data updated and saved.
Successfully authenticated and accessed the sheet.
Data saved to Google Sheet.
Scheduled job to run every 30 mins...
Script interrupted by user, exiting.


In [ ]:
data_test = pd.read_csv("dogs_at_shelter.csv")
data_test = pd.DataFrame(data_test)

output = wrangle(data_test)